In [14]:
## read the file (.pairs)

In [1]:
import numpy.random as random
import numpy as np
import math
import networkx as nx
from scipy import sparse as sp
import random

In [149]:

## need to fix something


def read_pairs_file(pairfile) :
    
    file = open(pairfile, "r")
    
    edges = []
    weights = []
    
    for line in file :
        nodes = line.split()
        edges.append((nodes[0], nodes[1]))
        
        #weight
        if len(nodes) == 3 :
            weights.append(float(nodes[2]))
        else :
            weights.append(1.0)
    file.close()
    
    pair_dict = {}
    t_num = 0 
    
    for edge in edges :
        if edge[0] not in pair_dict :
            pair_dict[edge[0]] = t_num
            t_num +=1
        if edge[1] not in pair_dict :
            pair_dict[edge[1]] = t_num
            t_num +=1
    
    
    adj =[]
    
    adj.extend([[pair_dict[edge[0]],pair_dict[edge[1]]] for edge in edges])
    adj.extend([[pair_dict[edge[1]],pair_dict[edge[0]]] for edge in edges])
    
    print(adj)
    adj = np.array(adj).transpose()
    
    weights.extend(weights)
    w_mat = np.array(weights)
    
    n = t_num  # number of nodes
    #print(n)
    #print(w_mat)
    #print(adj)
    
    A_mat = sp.csr_matrix((w_mat,adj), shape = (n,n))
    
    # number of nodes 
    n_A = A_mat
    sumit  = A_mat.sum()
    m = (sumit/2) # number of edge
    return n,n_A ,m

## we need network

##read from matrix file

def read_adj_file(adjfile) :
    #read matrix file 
    
    file = open(adjfile, 'r')
    cols = file.readline().split()
    
    #number of nodes
    n = len(cols)
   
    #generanate NXN matrix
    mat = sp.lil_matrix((n, n))
    
    
    #splited first row 
    for idx in range(n):
        
        weight = float(cols[idx])
        if weight != 0 :
            mat[0,idx] = weight
    
    #rest of matrix 
    
    r = 1
    for line in file :
        rows= line.split()
        for idx in range(n) :
            weight = float(rows[idx])
            if weight !=0  :
                mat[r,idx] = weight
            
        r +=1
        
    mat = (mat +matrix.T)/2
    
    #number of edges
    
    
    file.close()
    
    m = (mat.sum())/2
    
    return n, mat, m
        

In [157]:
n, n_A, m=read_adj_file("karate.txt")

In [158]:
n_A.toarray()

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [166]:
#n is number of elements, cn, constrainumber
def initial_grouping(n, cn):

    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign

#partition is group-assigned list
def partition_matrix(partition) :
    n = len(partition)
    data= np.ones(n)
    #print(data)
    ij = np.array([partition, list(range(0,n))])
    #print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

# s = partition matrix, k = degree matrix , m = number of edge 

#generating degree matrix 
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat


def modularity_mat(adj, k, m ):
    
    null_model  = np.dot(k,k.transpose())/(2*m)
    m_m= (adj - null_model)
    
    return m_m

#calculate modularity
def get_modularity(adj, s, k, m):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    #adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    #deg_par = np.array(s*k)
    #degree_partition = np.square(deg_par).sum()/(2*m) 
    #modularity
    #Q = (adj_partition - degree_partition) /(2*m)
    
    mod_matrix = modularity_mat(adj, k, m )
    modul = (s*mod_matrix*s.transpose()).diagonal().sum()
    Q = modul/(2*m)
    
    return Q

def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group



def annealing_simulating(initial_temp, adj,n, cn,  s, k, m, cooling_constant, iteration) :
    
    svg_group_list = []
    modul_list = []
    old_Q = None
    group_list = None
    for i in range(iteration) :
        
        temp = new_temp(cooling_constant, initial_temp, i)
        acceptance = False 
        num_rejection = -1
        #n numberof nodes, cn constrained number of groups
        rej_threshold = cal_rej_thres(n,cn)
        #print(rej_threshold)
        
        while acceptance is False :
            
            num_rejection +=1
            #print(old_Q)
            
            #when the very first
            if old_Q is None :
                node_list , group_list = initial_grouping(n,cn)
                s = partition_matrix(group_list)
                old_Q = get_modularity(adj, s, k, m)
                
            
            # patching new grouping list
            new_grouping = patching(group_list, cn)
            #print(new_grouping)
            
            # new grouping matrix
            new_s = partition_matrix(new_grouping)
            
            # new modularity with new grouping matrix
            new_Q = get_modularity(adj,new_s,k,m)
            #print(new_Q)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            #print(acceptance)
            #print(num_rejection)
            if num_rejection > rej_threshold:
                break
            
            
            if acceptance is True :
                
                #if new_grouping in svg_group_list :
                    #acceptance = False
                #else :
                svg_group_list.append(new_grouping)
                modul_list.append(new_Q)
                print(modul_list[-1])
                old_Q = new_Q
                # accepted grouping would be old grouping and update group based on this grouping 
                group_list = new_grouping
                
    opt = max(modul_list)
    idx_opt = modul_list.index(opt)
    
    opt_group = svg_group_list[idx_opt]
    
    opt_grouping = svg_format(opt_group) 
    opt_Q = opt
    
    return opt_grouping, opt_Q, svg_group_list, modul_list
            


# get updated temperature
def new_temp(cooling_constant, prev_temp, times) :
    new_temp = (math.pow(cooling_constant,times)) * prev_temp
    return new_temp 

# check acceptance between old groups and new groups
def check_acceptance(modul_old, modul_new  , temp) :
    if modul_new > modul_old :
        return True
    else :
        new_accept = math.exp((modul_new - modul_old) * temp)
        #modul_new- modul_old is negative value
        # when temp is higher , new_accept is getting smaller 
    return (np.random.random_sample() < new_accept)  #get true or false



#calculate rejection threshold 

def cal_rej_thres(n, nc):
    
    num_pathing = n*nc
    
    num_of_moves = num_pathing
    
    confi = 1 - math.log(0.05)/math.log(num_of_moves)
    
    return confi *num_of_moves * math.log(num_of_moves)

#saving format for check group easily 

def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set



## Simulated Annealing for modularity optimization (Operating code)

In [167]:
#n, n_A, m=read_pairs_file("karate.pairs")
n,n_A,m = read_adj_file('sourcefile/karate.adj')
k = degree_matrix(n_A)
#(initial_temp, adj, s, k, m, cooling_constant, iteration)
opt_grouping, opt_Q, saving_group, modul_list = annealing_simulating(initial_temp = 40, adj = n_A,n =n, cn=2,  s =s, k =k,m= m, cooling_constant = 1.0015, iteration =10000) 

-0.013806706114398465
-0.008218277449046697
0.0009861932938856437
0.004602235371466124
0.026627218934911285
0.024901380670611453
-8.218277449045512e-05
-0.026380670611439828
-0.03879026955950037
-0.03879026955950037
-0.03977646285338592
-0.038461538461538464
-0.03879026955950037
-0.041420118343195304
-0.043721236028928315
-0.041420118343195304
-0.03879026955950037
-0.03879026955950037
-0.01577909270216966
-0.013149243918474732
-0.01282051282051282
-0.026380670611439828
-0.025723208415516095
-0.025723208415516095
-0.014135437212360285
-0.018080210387902678
0.0009861932938856437
-0.028928336620644302
-0.009944115713346528
-0.018080210387902678
-0.0020545693622616512
-0.025723208415516095
-0.025723208415516095
0.017422748191978945
0.06278763971071664
0.05119986850756083
0.007149901380670598
-0.017094017094017123
-0.001314924391847464
0.001890203813280741
0.02358645627876399
0.04602235371466142
0.05687047994740305
0.04922748191978963
0.07659434582511501
0.05687047994740305
0.13535502958579

0.016765285996055212
0.005588428665351768
0.011834319526627177
0.009861932938855981
0.011505588428665356
0.024901380670611453
-8.218277449045512e-05
-0.0020545693622616512
-0.006656804733727775
-0.018491124260355044
0.021614069690992794
0.017422748191978945
0.030243261012491765
0.00780736357659433
0.008547008547008517
0.01577909270216966
0.015039447731755381
0.01577909270216966
0.03418803418803416
-0.001314924391847464
0.005588428665351768
0.024326101249178177
0.03122945430637741
0.02753122945430638
0.04799474030243262
-0.04413214990138069
0.012491781722550908
0.025558842866535186
0.025558842866535186
0.025558842866535186
0.024901380670611453
0.025558842866535186
0.004602235371466124
0.011505588428665356
0.012491781722550908
0.03813280736357655
0.037146614069691
0.033201840894148606
0.055884286653517405
0.03344838921761997
0.05687047994740305
0.07363576594345826
0.08941485864562783
0.12417817225509535
0.12746548323471402
0.12812294543063776
0.13280736357659434
0.15179158448389218
0.178

In [168]:
opt_grouping

{1: [0,
  1,
  4,
  7,
  8,
  10,
  11,
  13,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  29,
  30,
  32,
  33],
 0: [2, 3, 5, 6, 9, 12, 14, 15, 25, 26, 27, 28, 31]}

## Simulated Annealing

In [160]:
def annealing_simulating(initial_temp, adj,n, cn,  s, k, m, cooling_constant, iteration) :
    
    svg_group_list = []
    modul_list = []
    old_Q = None
    group_list = None
    for i in range(iteration) :
        
        temp = new_temp(cooling_constant, initial_temp, i)
        acceptance = False 
        num_rejection = -1
        #n numberof nodes, cn constrained number of groups
        rej_threshold = cal_rej_thres(n,cn)
        #print(rej_threshold)
        
        while acceptance is False :
            
            num_rejection +=1
            #print(old_Q)
            
            #when the very first
            if old_Q is None :
                node_list , group_list = initial_grouping(n,cn)
                s = partition_matrix(group_list)
                old_Q = get_modularity(adj, s, k, m)
                
            
            # patching new grouping list
            new_grouping = patching(group_list, cn)
            #print(new_grouping)
            
            # new grouping matrix
            new_s = partition_matrix(new_grouping)
            
            # new modularity with new grouping matrix
            new_Q = get_modularity(adj,new_s,k,m)
            #print(new_Q)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            #print(acceptance)
            #print(num_rejection)
            if num_rejection > rej_threshold:
                break
            
            
            if acceptance is True :
                
                #if new_grouping in svg_group_list :
                    #acceptance = False
                #else :
                svg_group_list.append(new_grouping)
                modul_list.append(new_Q)
                #print(modul_list[-1])
                old_Q = new_Q
                group_list = new_grouping
                
    opt = max(modul_list)
    idx_opt = modul_list.index(opt)
    
    opt_group = svg_group_list[idx_opt]
    
    opt_grouping = svg_format(opt_group) 
    opt_Q = opt
    
    return opt_grouping, opt_Q, svg_group_list, modul_list
            
        
        

## Getting modularity 

In [183]:
# s = partition matrix, k = degree matrix , m = number of edge 
m = (sumit/2)

def get_modularity(adj, s, k, m ):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    
    adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    deg_par = np.array(s*k)
    degree_partition = np.square(deg_par).sum()/(2*m) 
    
    #modularity
    Q = (adj_partition - degree_partition) /(2*m)
    
    return Q
    

78.0


## Degree matrix

In [ ]:
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat

## Initial grouping

In [11]:
#n is number of elements, cn, constrainumber
def initial_grouping(n,cn):
    
    
    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign
    

## Getting S(Partition matrix) 

In [15]:
def partition_matrix(partition) :
    print(partition)
    n = len(partition)
    data= np.ones(n)
    print(data)
    ij = np.array([partition, list(range(0,n))])
    print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

## Saving format for checking group easily

In [172]:
def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set

## Pathcing the group move singe node into other groups

In [1]:
def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group
    